In [106]:
# Some Utils and general functions

from secrets import token_hex, randbelow
import random
import plotly.graph_objects as go
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import imageio.v2 as imageio
import os
import pickle
from math import floor
import multiprocessing


def gen_node_id():
    return token_hex(32)

def rand_percent():
    # Generate a random integer within the specified range [min_value, max_value]
    random_int = randbelow(100000)
    return random_int / (100000)

# For 3d Models to gif
def generate_frame(angle, vertical_angle, pov_distance, frame_path, fig):
    up_vector = [0, 0, np.cos(np.radians(vertical_angle))]
    up_vector /= np.linalg.norm(up_vector)

    eye_vector = [
        pov_distance * np.cos(np.radians(angle)),
        pov_distance * np.sin(np.radians(angle)),
        pov_distance * np.sin(np.radians(vertical_angle))
    ]

    fig.update_layout(scene_camera=dict(eye=dict(x=eye_vector[0], y=eye_vector[1], z=eye_vector[2]), up=dict(x=up_vector[0], y=up_vector[1], z=up_vector[2])))
    fig.write_image(frame_path)


def calculate_small_worldness(graph):
    # Step 1: Calculate average shortest path length (L)
    L = nx.average_shortest_path_length(graph)
    
    # Step 2: Calculate average clustering coefficient (C)
    C = nx.average_clustering(graph)
    
    # Step 3: Generate a random graph with the same number of nodes and approximate number of edges
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()
    
    # Calculate the average degree in the original graph
    avg_degree = 2 * num_edges / num_nodes
    
    # Generate a random graph with the same number of nodes and approximate number of edges
    random_graph = nx.connected_watts_strogatz_graph(num_nodes, round(avg_degree), 0.1)

    # Calculate its L_rand and C_rand
    L_rand = nx.average_shortest_path_length(random_graph)
    C_rand = nx.average_clustering(random_graph)
    
    # Step 4: Calculate small-worldness (σ)
    try:
        sigma = (C / C_rand) / (L / L_rand)
    except:
        sigma = 0
    return sigma

def small_world_index(graph_original):

    num_nodes = graph_original.number_of_nodes()
    num_edges = graph_original.number_of_edges()
    avg_degree = 2 * num_edges / num_nodes
    graph_randomized = nx.connected_watts_strogatz_graph(num_nodes, round(avg_degree), 0.1)

    # Calculate the number of nodes and edges in the original graph
    n = len(graph_original.nodes)
    m = len(graph_original.edges)

    # Calculate the characteristic path length (C) and clustering coefficient (L) for the original graph
    C = nx.average_clustering(graph_original)
    L = nx.average_shortest_path_length(graph_original)

    # Calculate the characteristic path length (C_r) and clustering coefficient (L_r) for the randomized graph
    C_r = nx.average_clustering(graph_randomized)
    L_r = nx.average_shortest_path_length(graph_randomized)

    # Calculate the characteristic path length (C_ell) and clustering coefficient (L_ell) for an equivalent lattice graph
    lattice_graph = nx.watts_strogatz_graph(n, k=4, p=0)
    C_ell = nx.average_clustering(lattice_graph)
    L_ell = nx.average_shortest_path_length(lattice_graph)

    # Calculate the small-world index
    small_world_index = ((L - L_ell) / (L_r - L_ell)) * ((C - C_r) / (C_ell - C_r))

    return small_world_index

def random_swn_stats(graph):
    # Step 3: Generate a random graph with the same number of nodes and approximate number of edges
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()
    
    # Calculate the average degree in the original graph
    avg_degree = 2 * num_edges / num_nodes
    #print(f'NODES: {num_nodes}')
    #print(f'EDGES: {num_edges}')
    #print(f'AVG DEGREE: {avg_degree}')
    
    # Generate a random graph with the same number of nodes and approximate number of edges
    random_graph = nx.connected_watts_strogatz_graph(num_nodes, round(avg_degree), 0.1)
    #print(f'RAND NODES: {random_graph.number_of_nodes()}')
    #print(f'RAND EDGES: {random_graph.number_of_edges()}')

    # Calculate its L_rand and C_rand
    L_rand = nx.average_shortest_path_length(random_graph)
    C_rand = nx.average_clustering(random_graph)

    return L_rand, C_rand

def save_model_data(model, basename):
    model.generate_model_2d()
    model.generate_model_3d()
    model.save_to_file(f'{basename}.pkl')

    #model.update_model_3d(mode='betweenness')
    #model.update_model_2d(mode='betweenness')
    #model.model_3d_to_gif(f'{basename}_betweenness.gif')
    #model.model_2d_to_png(f'{basename}_betweenness.png')

    model.update_model_3d(mode='katz')
    model.update_model_2d(mode='katz')
    model.model_3d_to_gif(f'{basename}_katz.gif')
    model.model_2d_to_png(f'{basename}_katz.png')

def save_stress_model_data(model, per, basename):
    model.generate_model_2d()
    model.generate_model_3d()
    model.save_to_file(f'{basename}.pkl')

    model.update_model_3d(mode='betweenness')
    model.update_model_2d(mode='betweenness')
    title = f"Resiliency of a SWN with Connection Protocol (Oldest {per}% Removed)"
    model.update_3d_title(title)
    model.update_2d_title(title)
    model.model_3d_to_gif(f'{basename}_betweenness.gif')
    model.model_2d_to_png(f'{basename}_betweenness.png')

    model.update_model_3d(mode='katz')
    model.update_model_2d(mode='katz')
    title = f"Resiliency of a SWN with Connection Protocol (Oldest {per}% Removed)"
    model.update_3d_title(title)
    model.update_2d_title(title)
    model.model_3d_to_gif(f'{basename}_katz.gif')
    model.model_2d_to_png(f'{basename}_katz.png')

def plot_scatter(x_y_tuples, title, x_axis_label, y_axis_label, filename):
    #Parameters:
    #x_y_tuples (list of tuples): An array of X/Y tuples.

    # Unzip the tuples into separate X and Y arrays
    x, y = zip(*x_y_tuples)
    #plt.scatter(x, y, label='Scatter Plot', color='b', marker='o')
    plt.scatter(x, y, color='b', marker='o')
    plt.xlabel(x_axis_label)
    plt.ylabel(y_axis_label)
    plt.title(title)

    # Fit a quadratic (2nd-degree polynomial) trendline to the data
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    trendline_x = np.linspace(min(x), max(x), 100)
    trendline_y = p(trendline_x)

    # Plot the trendline
    plt.plot(trendline_x, trendline_y, label='Trend Line', color='r')
    plt.legend() # Plot the legend
    plt.savefig(filename, format='png')
    plt.show()

def plot_multicolored_lines_labels(lines, title, labels, x_axis_label, y_axis_label, filename):
    # (width, height)
    figsize=(1000, 800)
    dpi=100
    #Parameters:
    #lines (list of list of tuples): An array of lines, where each line is represented as an array of X/Y tuples.
    #labels (list of str): Custom labels for the legend.
    #filename (str): The name of the PNG file to save.
    #eg:
    #custom_labels = ["Line A", "Line B", "Line C"]
    #lines_data = [
    #    [(1, 2), (2, 4), (3, 6), (4, 8), (5, 10)],
    #    [(1, 1), (2, 3), (3, 7), (4, 9), (5, 12)],
    #    [(1, 3), (2, 5), (3, 8), (4, 11), (5, 15)]
    #]
    plt.figure(figsize=(figsize[0]/dpi, figsize[1]/dpi), dpi=dpi)
    for i, line in enumerate(lines):
        x, y = zip(*line)
        color = plt.cm.rainbow(i / len(lines)) # Generate a unique color for each line
        plt.plot(x, y, label=labels[i], color=color) # Create a line plot for the current line with a unique color
    plt.xlabel(x_axis_label)
    plt.ylabel(y_axis_label)
    plt.title(title)
    plt.legend()

    # Save the plot to a PNG file
    plt.savefig(filename, format='png', dpi=dpi)
    plt.show()

def plot_multicolored_lines(lines, title, x_axis_label, y_axis_label, filename):
    #Parameters:
    #lines (list of list of tuples): An array of lines, where each line is represented as an array of X/Y tuples.
    #labels (list of str): Custom labels for the legend.
    #filename (str): The name of the PNG file to save.
    #eg:
    #custom_labels = ["Line A", "Line B", "Line C"]
    #lines_data = [
    #    [(1, 2), (2, 4), (3, 6), (4, 8), (5, 10)],
    #    [(1, 1), (2, 3), (3, 7), (4, 9), (5, 12)],
    #    [(1, 3), (2, 5), (3, 8), (4, 11), (5, 15)]
    #]
    for i, line in enumerate(lines):
        x, y = zip(*line)
        color = plt.cm.rainbow(i / len(lines)) # Generate a unique color for each line
        plt.plot(x, y, label=f'Line {i+1}', color=color) # Create a line plot for the current line with a unique color
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.title('Line Graph with Multiple Colored Lines')
    plt.legend()

    # Save the plot to a PNG file
    plt.savefig(filename, format='png')
    plt.show()

In [ ]:
class Network:
    def __init__(self, MAX):
        #n = Node(MAX)
        self.MAX_NEIGHBORS = MAX
        #self.nodes = {
        #    n.id: n
        #}

        self.graph = nx.Graph()
        self.graph.add_node(
            gen_node_id(),
            max_neighbors=self.MAX_NEIGHBORS
        )
    def save_to_file(self, filename):
        with open(filename, 'wb') as file:
            pickle.dump(self, file)

    @classmethod
    def load_from_file(cls, filename):
        with open(filename, 'rb') as file:
            obj = pickle.load(file)
        return obj

    def replicate_node_og(self, node_id):
        connect_id = self.determine_connect(node_id)
        new_node_id = gen_node_id()
        self.graph.add_node(
            new_node_id,
            max_neighbors=self.MAX_NEIGHBORS
        )
        self.connect(new_node_id, connect_id)

    def propogate_og(self, iters):
        for i in range(iters):
            nodes = list(self.graph.__iter__())
            for node_id in nodes:
                self.replicate_node_og(node_id)
    
    def replicate_node(self, node_id):
        #new_node = Node(self.MAX_NEIGHBORS)

        new_node_id = gen_node_id()
        
        # Use Self and +2 (up to connect_id_3 total)

        connect_id_1 = None
        #if len( list(self.graph.neighbors(node_id)) ) ==  0:
        #    connect_id_1 = self.determine_connect(node_id)
        connect_id_1 = self.determine_connect(node_id)
        connect_id_2 = None
        if len( list(self.graph.neighbors(node_id)) ) > 1:
            connect_id_2 = self.determine_connect(list(self.graph.neighbors(node_id))[-2])
        connect_id_3 = None
        if len( list(self.graph.neighbors(node_id)) ) > 2:
            connect_id_3 = self.determine_connect(list(self.graph.neighbors(node_id))[-3])
        connect_id_4 = None
        #if len( list(self.graph.neighbors(node_id)) ) > 3:
        #    connect_id_4 = self.determine_connect(list(self.graph.neighbors(node_id))[-4])

        self.graph.add_node(
            new_node_id,
            max_neighbors=self.MAX_NEIGHBORS,
        )

        if connect_id_1:
            self.connect(new_node_id, connect_id_1)
        if connect_id_2:
            self.connect(new_node_id, connect_id_2)
        if connect_id_3:
            self.connect(new_node_id, connect_id_3)
        if connect_id_4:
            self.connect(new_node_id, connect_id_4)

    def propogate(self, iters):
        for i in range(iters):
            #print(f"Iter: {i}")
            nodes = list(self.graph.__iter__())
            for node_id in nodes:
                self.replicate_node(node_id)
    
    def propogate_with_removal(self, iters):
        for i in range(iters):
            nodes = list(self.graph.__iter__())
            for node_id in nodes:
                if len(self.graph) > 1 and rand_percent() < 0.05:
                    self.remove_node(node_id)
            
            nodes = list(self.graph.__iter__())
            for node_id in nodes:
                self.replicate_node(node_id)

    def propogate_full_rand(self, graph):
        # Propogate a random smn of a size approximate to a given graph

        # Step 3: Generate a random graph with the same number of nodes and approximate number of edges
        num_nodes = graph.number_of_nodes()
        num_edges = graph.number_of_edges()
        
        # Calculate the average degree in the original graph
        avg_degree = 2 * num_edges / num_nodes
        
        # Generate a random graph with the same number of nodes and approximate number of edges
        self.graph = nx.connected_watts_strogatz_graph(num_nodes, round(avg_degree), 0.1)

        # Calculate its L_rand and C_rand
        L_rand = nx.average_shortest_path_length(self.graph)
        C_rand = nx.average_clustering(self.graph)

        return L_rand, C_rand

    def connect(self, node1, node2):
        self.graph.add_edge(node1, node2)

    def check_connection(self, node_id):
        code = None
        # 0 - Yes, can connect
        # 1 - No, but neighbors can
        # 2 - No, but here's the last (self.neighbors[-1])
        
        neighbors = list(self.graph.neighbors(node_id))

        if len(neighbors) < self.graph.nodes.get(node_id)['max_neighbors']:
            code = 0
        else:
            any_empty = False
            for neighbor_id in neighbors:
                if len(list(self.graph.neighbors(neighbor_id))) < self.graph.nodes.get(neighbor_id)['max_neighbors']:
                    any_empty = True
            if any_empty:
                code = 1
            else:
                code = 2

        return code
    
    def determine_connect(self, desired_id):
        connect_id = None
        while not connect_id:
            code = self.check_connection(desired_id)
            if code == 0:
                connect_id = desired_id
            elif code == 1:
                for neighbor_id in self.graph.neighbors(desired_id):
                    if len(list(self.graph.neighbors(neighbor_id))) < self.graph.nodes.get(neighbor_id)['max_neighbors']:
                        connect_id = neighbor_id
                        break
            else:
                desired_id = list(self.graph.neighbors(desired_id))[-1]
        return connect_id

    def remove_node(self, node_id):
        if self.graph.has_node(node_id):
            self.graph.remove_node(node_id)


    def remove_top_nodes(self, num):
        nodes = list(self.graph.__iter__())[:num]
        for node_id in nodes:
            self.remove_node(node_id)

    def stress_test(self):
        test_graph = self.graph.copy()
        i = 0
        nodes = list(self.graph.__iter__())

        while nx.is_connected(test_graph):
            test_graph.remove_node(nodes[i])
            i += 1
        return i-1
        


In [ ]:
# For modeling the networks in 2d and 3d

#==========================================================================
#Spring Layout: This is a force-directed layout algorithm 
# that simulates a physical system where nodes repel each other, 
# and edges act as springs. It is often used for general-purpose 
# network visualization.
#pos = nx.spring_layout(G)

#Kamada-Kawai Layout: This is another force-directed layout 
# algorithm that optimizes the placement of nodes based 
# on the distance between them. It tends to produce visually 
# pleasing layouts.
#pos = nx.kamada_kawai_layout(G, scale=2.0, center=[0, 0])

#Circular Layout: Places nodes in a circular arrangement, which 
# can be useful for visualizing cyclic or radial structures.
#pos = nx.circular_layout(G)

#Spectral Layout: Uses spectral graph theory to position nodes 
# in a way that preserves the graph's spectral properties.
#pos = nx.spectral_layout(G)

#Fruchterman-Reingold Layout: Another force-directed layout 
# algorithm that balances the forces between nodes to 
# produce an aesthetically pleasing layout.
#pos = nx.fruchterman_reingold_layout(G)

#Planar Layout: Attempts to position nodes in a way that 
# avoids edge crossings, making it suitable for planar graphs.
#pos = nx.planar_layout(G)

#Shell Layout: Arranges nodes in concentric circles or shells, 
# which can be useful for some hierarchical network structures.
#pos = nx.shell_layout(G)

#Random Layout: Positions nodes randomly within a specified area.
#pos = nx.random_layout(G)
#==========================================================================
# Degree Centrality:
#    Degree centrality of a node is the number of edges connected to that node.
#    It measures how well-connected a node is and can represent its popularity or influence.

# Betweenness Centrality:
#    Betweenness centrality measures how often a node lies on the shortest path between other nodes.
#    It identifies nodes that act as bridges between different parts of the network and play a critical role in information flow.

# Eigenvector Centrality:
#    Eigenvector centrality is based on the concept that important nodes are connected to other important nodes.
#    It assigns a centrality score to each node based on its connections to high-scoring nodes, which can uncover nodes with influence in a network.

# Katz Centrality:
#    Katz centrality is an extension of degree centrality that also considers the influence of nodes with higher centrality.
#    It assigns centrality scores based on the number of paths connecting a node to others, with diminishing returns for longer paths.
#==========================================================================

class NetworkModel:
    def __init__(self, network):
        self.graph = network.graph
        self.model_3d = None
        self.model_2d = None

        self.max_connections = network.MAX_NEIGHBORS

        self.cluster_co = "{:.3f}".format(nx.average_clustering(self.graph) )
        self.swi = "{:.3f}".format(calculate_small_worldness(network.graph))
        #self.avg_shortest = "{:.3f}".format(nx.average_shortest_path_length(self.graph))# Calculate the average shortest path

    def save_to_file(self, filename):
        with open(filename, 'wb') as file:
            pickle.dump(self, file)

    @classmethod
    def load_from_file(cls, filename):
        with open(filename, 'rb') as file:
            obj = pickle.load(file)
        return obj

    def num_nodes(self):
        return len(self.graph)
    
    def num_edges(self):
        return self.graph.number_of_edges()

    def generate_model_3d(self, mode='connections'):
        # Create a NetworkX graph from the network data
        if self.model_3d:
            return
        
        #self.graph = self.graph

        # Kamada-Kawai Layout with 3D
        pos = nx.kamada_kawai_layout(self.graph, dim=3)  # Use 3 dimensions
        #pos = nx.circular_layout(self.graph, dim=3)
        #pos = nx.fruchterman_reingold_layout(self.graph, dim=3)

        # Create a 3D network graph
        edge_x = []
        edge_y = []
        edge_z = []

        for edge in self.graph.edges():
            x0, y0, z0 = pos[edge[0]]
            x1, y1, z1 = pos[edge[1]]
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])
            edge_z.extend([z0, z1, None])

        edge_trace = go.Scatter3d(
            x=edge_x, y=edge_y, z=edge_z,
            line=dict(width=1, color='#333'),
            hoverinfo='none',
            mode='lines')

        node_x = []
        node_y = []
        node_z = []

        for node_id in self.graph.nodes():
            x, y, z = pos[node_id]
            node_x.append(x)
            node_y.append(y)
            node_z.append(z)

        if mode == 'connections':
            title = 'Node Connections'
            cscale = 'YlGnBu'
            reverse = True
        elif mode == 'betweenness':
            title = 'Betweenness Centrality'
            cscale = 'thermal'
            reverse = False
        elif mode == 'eigenvector':
            title = 'Eigenvector Centrality'
            cscale = 'ice'
            reverse = False
        elif mode == 'katz':
            title = 'Katz Centrality'
            cscale = 'agsunset'
            reverse = False

        node_trace = go.Scatter3d(
            x=node_x, y=node_y, z=node_z,
            mode='markers',
            hoverinfo='text',
            marker=dict(
                showscale=True,
                colorscale=cscale,
                reversescale=reverse,
                color=[],
                size=8,
                colorbar=dict(
                    thickness=15,
                    title=title,
                    xanchor='left',
                    titleside='right'
                ),
                line_width=2))

        # Color Node Points based on mode
        node_values = []
        node_text = []

        if mode == 'connections':
            for node_id in self.graph.nodes():
                node_values.append(len(list(self.graph.neighbors(node_id))))
                node_text.append(f'Node ID: {node_id}<br># of connections: {len(list(self.graph.neighbors(node_id)))}')
        elif mode == 'betweenness':
            betweenness_centrality = nx.betweenness_centrality(self.graph)
            for node_id in self.graph.nodes():
                node_values.append(betweenness_centrality[node_id])
                node_text.append(f'Node ID: {node_id}<br>Betweenness Centrality: {betweenness_centrality[node_id]}')
        elif mode == 'eigenvector':
            eigenvector_centrality = nx.eigenvector_centrality(self.graph)
            for node_id in self.graph.nodes():
                node_values.append(eigenvector_centrality[node_id])
                node_text.append(f'Node ID: {node_id}<br>Eigenvector Centrality: {eigenvector_centrality[node_id]}')
        elif mode == 'katz':
            katz_centrality = nx.katz_centrality(self.graph)
            for node_id in self.graph.nodes():
                node_values.append(katz_centrality[node_id])
                node_text.append(f'Node ID: {node_id}<br>Katz Centrality: {katz_centrality[node_id]}')

        
        node_trace.marker.color = node_values
        node_trace.text = node_text
        
        fig = go.Figure(data=[edge_trace, node_trace],
                        layout=go.Layout(
                            #title=f"<br><b>{title} of a SWN with Connection Protocol</b><br>Conns. (10%/Node of +1) : {self.max_connections}<br>Cluster Co. : {self.cluster_co}<br>Avg. Shortest : {self.avg_shortest}",
                            #title=f"<br><b>{title} of a SWN with Connection Protocol</b><br>Conns. : {self.max_connections}<br>Cluster Co. : {self.cluster_co}",
                            title=f"<br><b>{title} of a SWN with Connection Protocol</b><br>Conns. : {self.max_connections}<br>\u03C3 : {self.swi}",
                            titlefont_size=16,
                            showlegend=False,
                            hovermode='closest',
                            margin=dict(b=20, l=5, r=5, t=40),  # Adjust these values for the margin (b=bottom, l=left, r=right, t=top)
                            scene=dict(
                                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                                zaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
                            )
                        ))
        fig.update_layout(width=1000, height=1000)  # Set the width and height as per your preference

        self.model_3d = fig


    def update_model_3d(self, mode='connections'):
        # Update the mode and color scheme based on the new mode
        if mode == 'connections':
            title = 'Node Connections'
            cscale = 'YlGnBu'
            reverse = True
        elif mode == 'betweenness':
            title = 'Betweenness Centrality'
            cscale = 'thermal'
            reverse = False
        elif mode == 'eigenvector':
            title = 'Eigenvector Centrality'
            cscale = 'ice'
            reverse = False
        elif mode == 'katz':
            title = 'Katz Centrality'
            cscale = 'agsunset'
            reverse = False
        elif mode == 'clustering':
            title = 'Clustering Coefficient'
            cscale = 'viridis'  # You can choose a different color scale
            reverse = False
        

        # Update node_trace based on the new mode
        node_values = []
        node_text = []

        if mode == 'connections':
            for node_id in self.graph.nodes():
                node_values.append(len(list(self.graph.neighbors(node_id))))
                node_text.append(f'Node ID: {node_id}<br># of connections: {len(list(self.graph.neighbors(node_id)))}')
        elif mode == 'betweenness':
            betweenness_centrality = nx.betweenness_centrality(self.graph)
            for node_id in self.graph.nodes():
                node_values.append(betweenness_centrality[node_id])
                node_text.append(f'Node ID: {node_id}<br>Betweenness Centrality: {betweenness_centrality[node_id]}')
        elif mode == 'eigenvector':
            eigenvector_centrality = nx.eigenvector_centrality(self.graph)
            for node_id in self.graph.nodes():
                node_values.append(eigenvector_centrality[node_id])
                node_text.append(f'Node ID: {node_id}<br>Eigenvector Centrality: {eigenvector_centrality[node_id]}')
        elif mode == 'katz':
            katz_centrality = nx.katz_centrality(self.graph)
            for node_id in self.graph.nodes():
                node_values.append(katz_centrality[node_id])
                node_text.append(f'Node ID: {node_id}<br>Katz Centrality: {katz_centrality[node_id]}')
        elif mode == 'clustering':
            clustering_coefficients = nx.clustering(self.graph)
            for node_id in self.graph.nodes():
                node_values.append(clustering_coefficients[node_id])
                node_text.append(f'Node ID: {node_id}<br>Clustering Coefficients: {clustering_coefficients[node_id]}')

        self.model_3d.data[1].marker.colorscale = cscale
        self.model_3d.data[1].marker.reversescale = reverse
        self.model_3d.data[1].marker.color = node_values
        self.model_3d.data[1].marker.colorbar = dict(
            thickness=15,
            title=title,
            xanchor='left',
            titleside='right'
        )

        self.model_3d.data[1].text = node_text

        # Update the title of the layout
        #self.model_3d.update_layout(title=f"<br><b>{title} of a SWN with Connection Protocol</b><br>Conns. : {self.max_connections}<br>Cluster Co. : {self.cluster_co}<br>Avg. Shortest : {self.avg_shortest}")
        self.model_3d.update_layout(title=f"<br><b>{title} of a SWN with Connection Protocol</b><br>Conns. : {self.max_connections}<br>\u03C3 : {self.swi}")

        # Optionally, you can update other layout parameters if needed
        # self.model_3d.update_layout(...)

        return self.model_3d

    def generate_model_2d(self, mode='connections'):
        # Create a NetworkX graph from the network data
        if self.model_2d:
            return

        #self.graph = self.graph

        # Kamada-Kawai Layout with 2D
        pos = nx.kamada_kawai_layout(self.graph, dim=2)  # Use 2 dimensions

        # Create a 2D network graph
        edge_x = []
        edge_y = []

        for edge in self.graph.edges():
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])

        edge_trace = go.Scatter(
            x=edge_x, y=edge_y,
            line=dict(width=1, color='#444'),
            hoverinfo='none',
            mode='lines')

        node_x = []
        node_y = []

        for node_id in self.graph.nodes():
            x, y = pos[node_id]
            node_x.append(x)
            node_y.append(y)

        if mode == 'connections':
            title = 'Node Connections'
            cscale = 'YlGnBu'
            reverse = True
        elif mode == 'betweenness':
            title = 'Betweenness Centrality'
            cscale = 'thermal'
            reverse = False
        elif mode == 'eigenvector':
            title = 'Eigenvector Centrality'
            cscale = 'ice'
            reverse = False
        elif mode == 'katz':
            title = 'Katz Centrality'
            cscale = 'agsunset'
            reverse = False

        node_trace = go.Scatter(
            x=node_x, y=node_y,
            mode='markers',
            hoverinfo='text',
            marker=dict(
                showscale=True,
                colorscale=cscale,
                reversescale=reverse,
                color=[],
                size=8,
                colorbar=dict(
                    thickness=15,
                    title=title,
                    xanchor='left',
                    titleside='right'
                ),
                line_width=2))

        # Color Node Points based on mode
        node_values = []
        node_text = []

        if mode == 'connections':
            for node_id in self.graph.nodes():
                node_values.append(len(list(self.graph.neighbors(node_id))))
                node_text.append(f'Node ID: {node_id}<br># of connections: {len(list(self.graph.neighbors(node_id)))}')
        elif mode == 'betweenness':
            betweenness_centrality = nx.betweenness_centrality(self.graph)
            for node_id in self.graph.nodes():
                node_values.append(betweenness_centrality[node_id])
                node_text.append(f'Node ID: {node_id}<br>Betweenness Centrality: {betweenness_centrality[node_id]}')
        elif mode == 'eigenvector':
            #eigenvector_centrality = nx.eigenvector_centrality(self.graph)
            eigenvector_centrality = nx.normalized_eigenvector_centrality(self.graph)
            for node_id in self.graph.nodes():
                node_values.append(eigenvector_centrality[node_id])
                node_text.append(f'Node ID: {node_id}<br>Eigenvector Centrality: {eigenvector_centrality[node_id]}')
        elif mode == 'katz':
            katz_centrality = nx.katz_centrality(self.graph)
            for node_id in self.graph.nodes():
                node_values.append(katz_centrality[node_id])
                node_text.append(f'Node ID: {node_id}<br>Katz Centrality: {katz_centrality[node_id]}')

        node_trace.marker.color = node_values
        node_trace.text = node_text

        title += f"{title} of a SWN with Connection Protocol"

        fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                #title=f"<br><b>{title}</b><br>Conns. (10% of +1) : {self.max_connections}<br>Cluster Co. : {self.cluster_co}<br>Avg. Shortest : {self.avg_shortest}",
                title=f"<br><b>{title} of a SWN with Connection Protocol</b><br>Conns. : {self.max_connections}<br>\u03C3 : {self.swi}",
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20, l=5, r=5, t=40),  # Adjust these values for the margin (b=bottom, l=left, r=right, t=top)
                annotations=[dict(
                    text="",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002)],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

        fig.update_layout(width=1000, height=1000)  # Set the width and height as per your preference

        self.model_2d = fig

    def update_model_2d(self, mode='connections'):
        # Update the mode and color scheme based on the new mode
        if mode == 'connections':
            title = 'Node Connections'
            cscale = 'YlGnBu'
            reverse = True
        elif mode == 'betweenness':
            title = 'Betweenness Centrality'
            cscale = 'thermal'
            reverse = False
        elif mode == 'eigenvector':
            title = 'Eigenvector Centrality'
            cscale = 'ice'
            reverse = False
        elif mode == 'katz':
            title = 'Katz Centrality'
            cscale = 'agsunset'
            reverse = False
        elif mode == 'clustering':
            title = 'Clustering Coefficient'
            cscale = 'viridis'  # You can choose a different color scale
            reverse = False



        # Update node_trace based on the new mode
        node_values = []
        node_text = []

        if mode == 'connections':
            for node_id in self.graph.nodes():
                node_values.append(len(list(self.graph.neighbors(node_id))))
                node_text.append(f'Node ID: {node_id}<br># of connections: {len(list(self.graph.neighbors(node_id)))}')
        elif mode == 'betweenness':
            betweenness_centrality = nx.betweenness_centrality(self.graph)
            for node_id in self.graph.nodes():
                node_values.append(betweenness_centrality[node_id])
                node_text.append(f'Node ID: {node_id}<br>Betweenness Centrality: {betweenness_centrality[node_id]}')
        elif mode == 'eigenvector':
            eigenvector_centrality = nx.eigenvector_centrality(self.graph)
            for node_id in self.graph.nodes():
                node_values.append(eigenvector_centrality[node_id])
                node_text.append(f'Node ID: {node_id}<br>Eigenvector Centrality: {eigenvector_centrality[node_id]}')
        elif mode == 'katz':
            katz_centrality = nx.katz_centrality(self.graph)
            for node_id in self.graph.nodes():
                node_values.append(katz_centrality[node_id])
                node_text.append(f'Node ID: {node_id}<br>Katz Centrality: {katz_centrality[node_id]}')
        elif mode == 'clustering':
            clustering_coefficients = nx.clustering(self.graph)
            for node_id in self.graph.nodes():
                node_values.append(clustering_coefficients[node_id])
                node_text.append(f'Node ID: {node_id}<br>Clustering Coefficients: {clustering_coefficients[node_id]}')

        self.model_2d.data[1].marker.colorscale = cscale
        self.model_2d.data[1].marker.reversescale = reverse
        self.model_2d.data[1].marker.color = node_values
        self.model_2d.data[1].marker.colorbar = dict(
            thickness=15,
            title=title,
            xanchor='left',
            titleside='right'
        )

        self.model_2d.data[1].text = node_text

        # Update the title of the layout
        #self.model_2d.update_layout(title=f"<br><b>{title} of a SWN with Connection Protocol</b><br>Conns. (10%/Node of +1) : {self.max_connections}<br>Cluster Co. : {self.cluster_co}<br>Avg. Shortest : {self.avg_shortest}")
        self.model_2d.update_layout(title=f"<br><b>{title} of a SWN with Connection Protocol</b><br>Conns. : {self.max_connections}<br>\u03C3 : {self.swi}")
        
        # Optionally, you can update other layout parameters if needed
        # self.model_2d.update_layout(...)

        return self.model_2d

    def show_2d(self):
        self.model_2d.show()
        
    def show_3d(self):
        self.model_3d.show()
    
    def model_3d_to_gif(self, filename):
        num_rotations = 1
        #num_frames_per_rotation = 157
        num_frames_per_rotation = 130
        #num_frames_per_rotation = 99
        
        initial_vertical_angle_degrees = 20
        decrease_per_rotation = 40
        pov_distance = 1.5

        fig = self.model_3d
        layout = go.Layout(scene=dict(camera=dict(up=dict(x=0, y=0, z=1))))

        frame_paths = []
        frames = []

        vertical_angle = initial_vertical_angle_degrees

        processes = []
        num_processes = 8  # Set the number of processes

        for _ in range(num_rotations):
            angles = np.linspace(0, 360, num_frames_per_rotation, endpoint=False)

            for i in range(0, len(angles), num_processes):
                processes = []
                
                for angle in angles[i:i+num_processes]:
                    frame_path = f'frame_{angle}_{vertical_angle}.png'
                    frame_paths.append(frame_path)

                    process = multiprocessing.Process(target=generate_frame, args=(angle, vertical_angle, pov_distance, frame_path, fig))
                    processes.append(process)
                    process.start()
                    vertical_angle -= decrease_per_rotation / num_frames_per_rotation

                # Wait for all processes to complete
                for process in processes:
                    process.join()

        for frame in frame_paths:
            frames.append(imageio.imread(frame))

        # Create a GIF from the frames
        imageio.mimsave(filename, frames, loop=True)

        # Delete the individual frames
        for frame_path in frame_paths:
            os.remove(frame_path)

    def model_2d_to_png(self, filename):
        self.generate_model_2d()  # Ensure the figure is generated before saving
        self.model_2d.write_image(filename)
        print(f"Figure saved to {filename}")

    def update_3d_title(self, new_title):
        #self.model_3d.update_layout(title=f"<br><b>{new_title}</b><br>Conns. (10%/Node of +1) : {self.max_connections}<br>Cluster Co. : {self.cluster_co}<br>Avg. Shortest : {self.avg_shortest}")
        self.model_3d.update_layout(title=f"<br><b>{new_title} of a SWN with Connection Protocol</b><br>Conns. : {self.max_connections}<br>\u03C3 : {self.swi}")

    def update_2d_title(self, new_title):
        #self.model_2d.update_layout(title=f"<br><b>{new_title}</b><br>Conns. (10%/Node of +1) : {self.max_connections}<br>Cluster Co. : {self.cluster_co}<br>Avg. Shortest : {self.avg_shortest}")
        self.model_2d.update_layout(title=f"<br><b>{new_title} of a SWN with Connection Protocol</b><br>Conns. : {self.max_connections}<br>\u03C3 : {self.swi}")



In [108]:
# Some of the code used to generate networks and models
#------------------------------------------

network = Network(6)
network.propogate(12)
model = NetworkModel(network)


#model.generate_model_2d()
#model.update_model_2d(mode='katz')
#model.model_2d.update_layout(title=f"<br><b>Katz Centrality of a Star Topology</b>")
#model.show_2d()
#model.model_2d_to_png('6.png')

#model.generate_model_3d()
#model.update_model_3d(mode='katz')
#model.model_3d.update_layout(title=f"<br><b>Katz Centrality of a Star Topology</b>")
#model.show_3d()
#model.model_3d_to_gif('6.gif')

#network = Network(6)
#network.propogate(9)
#model = NetworkModel(network)
#model.generate_model_2d()
#model.update_model_2d(mode='katz')
#model.model_2d.update_layout(title=f"<br><b>Katz Centrality of a Star Topology</b>")
#model.show_2d()
#model.model_2d_to_png('bad.png')

#r = network.stress_test()
#print(r, network.graph.number_of_nodes(), (r/network.graph.number_of_nodes())*100)

#smodel = NetworkModel(network)
#smodel.generate_model_3d()
#smodel.update_model_3d(mode='katz')
#model.model_3d.update_layout(title=f"<br><b>Katz Centrality of a Star Topology</b>")
#smodel.show_3d()
#model.model_3d_to_gif('6.gif')

#------------------------------------------
#------------------------------------------
'''
network = Network(5)
network.propogate(7)
model = NetworkModel(network)
model.generate_model_2d()
model.update_model_2d(mode='katz')
model.model_2d.update_layout(title=f"<br><b>Katz Centrality of a Star Topology</b>")
model.show_2d()
model.model_2d_to_png('bad.png')

model.generate_model_3d()
model.update_model_3d(mode='katz')
model.model_3d.update_layout(title=f"<br><b>Katz Centrality of a Star Topology</b>")
model.show_3d()
model.model_3d_to_gif('bad.gif')
'''
#------------------------------------------
#------------------------------------------
'''
net = Network(6)
net.propogate(12)

network = Network(15)
network.propogate_full_rand(net.graph)
print('Generated. To models...')
model = NetworkModel(network)
model.generate_model_2d()
model.update_model_2d(mode='katz')
model.model_2d.update_layout(title=f"<br><b>Katz Centrality of a Watts-Strogatz Small-World Network</b><br>\u03C3 : {model.swi}")
model.show_2d()
model.model_2d_to_png('ws-swn.png')

model.generate_model_3d()
model.update_model_3d(mode='katz')
model.model_3d.update_layout(title=f"<br><b>Katz Centrality of a Watts-Strogatz Small-World Network</b><br>\u03C3 : {model.swi}")
model.show_3d()
model.model_3d_to_gif('ws-swn.gif')
'''
#------------------------------------------


#model.model_3d.update_layout(title=f"<br><b>Katz Centrality of Star Topology</b><br>Conns. : 4")
#model.model_3d_to_gif(basename+'.gif')

#print(4)
#model = NetworkModel(network)
#print(5)


#model.generate_model_2d()
#model.generate_model_3d()
#model.save_to_file(f'{basename}.pkl')

#model.update_model_3d(mode='betweenness')
#model.update_model_2d(mode='betweenness')
#model.model_3d_to_gif(f'{basename}_betweenness.gif')
#model.model_2d_to_png(f'{basename}_betweenness.png')

#model.update_model_3d(mode='katz')
#model.update_model_2d(mode='katz')
#model.model_3d_to_gif(f'{basename}_katz.gif')
#model.model_2d_to_png(f'{basename}_katz.png')

#save_model_data(model, basename)
#print(6)
#r = network.stress_test()
#print(7)
#smodel = NetworkModel(network)
#print(float("{:.3f}".format((r/network.graph.number_of_nodes()) * 100)))
#save_stress_model_data(smodel, float("{:.3f}".format((r/network.graph.number_of_nodes()) * 100)), basename)
#print(9)

#sigma = calculate_small_worldness(network.graph)
#print(f'\u03C3 : {sigma}')


#model.generate_model_3d()
#model.update_model_3d(mode='clustering')
#model.show_3d()

'''
labels = []
lines_cc = []
lines_sp = []
#connections = [6, 7, 8, 9, 10, 11, 12, 13, 14, 20, 25]
#connections = [6, 9, 12, 15, 18, 21, 24]
connections = [5, 6, 7, 8, 9]
for conns in connections: # 4, 13
    line_cc = []
    line_sp = []
    for iters in range(8, 12): # 6, 14
        print(conns, iters)
        avg_cc = 0
        avg_sp = 0
        for i in range(100):
            network = Network(conns)
            network.propogate(iters)
            avg_cc += nx.average_clustering(network.graph)
            avg_sp += nx.average_shortest_path_length(network.graph)
        x = 2**iters
        y_cc = avg_cc/100
        y_sp = avg_sp/100
        print('\t', y_sp)
        line_cc.append((x,y_cc))
        line_sp.append((x,y_sp))
    labels.append(f'{conns} Conns.')
    lines_cc.append(line_cc)
    lines_sp.append(line_sp)

pn = 0
plot_multicolored_lines_labels(
    lines_cc,
    #title = "Number of Nodes and Avg. Shortest Path Length",
    title = f"Number of Nodes and Avg. Cluster Coefficient",
    labels=labels,
    x_axis_label="Number of Nodes",
    #y_axis_label="Avg. Shortest Path Length",
    y_axis_label="Avg. Cluster Coefficient",
    filename=f"cluster_co_{pn}.png",
)
plot_multicolored_lines_labels(
    lines_sp,
    title = f"Number of Nodes and Avg. Shortest Path Length",
    #title = "Number of Nodes and Avg. Cluster",
    labels=labels,
    x_axis_label="Number of Nodes",
    y_axis_label="Avg. Shortest Path Length",
    #y_axis_label="Cluster Coefficient",
    filename=f"shortest_path_{pn}.png",
)
'''


'''
data = []
for iters in range(6, 13):
    for i in range(100):
        network = Network(15)
        network.propogate(iters)
        x = 2**iters
        y = nx.average_clustering(network.graph)
        data.append((x,y))

plot_scatter(
    data,
    title = "Number of Nodes and Cluster Coefficient",
    x_axis_label="Number of Nodes",
    y_axis_label="Cluster Coefficient",
    filename="test.png",
)
'''

'''
#for i in range(3):
#    network = Network(7)
#    network.propogate(12) # 9=512, 10=1024, 11=2048
#    model = NetworkModel(network)
#    save_model_data(model, f'7-12_10per-of-p1_{i}')
'''

'''
best_net = None
best_sur = 0
worst_net = None
worst_sur = 10000

for i in range(100):
    network = Network(7)
    network.propogate(12) # 9=512, 10=1024, 11=2048
    survivability = network.stress_test()

    if survivability > best_sur:
        best_net = network
        best_sur = survivability

    if survivability < worst_sur:
        worst_net = network
        worst_sur = survivability

#print()
print(best_sur)
#print(worst_sur)

best_model = NetworkModel(best_net)
worst_model = NetworkModel(worst_net)

print(best_model.cluster_co)
print(best_model.avg_shortest)

save_model_data(best_model, '11-10_10per-of-p2_best-sur')
#save_model_data(worst_model,'11-10_10per-of-p2_worst-sur')

total = len(best_net.graph)

best_net.remove_top_nodes(best_sur-1)
#worst_net.remove_top_nodes(worst_sur-1)

best_per = "{:.2f}".format((best_sur/total)*100)
#worst_per = "{:.2f}".format((worst_sur/total)*100)

best_stress_model = NetworkModel(best_net)
#worst_stress_model = NetworkModel(worst_net)

save_stress_model_data(best_stress_model, best_per, '7-12_10per-of-p1_best-sur_stress')
#save_stress_model_data(worst_stress_model, worst_per, '11-10_10per-of-p2_worst-sur_stress')
'''


NODES: 4096
EDGES: 12264
AVG DEGREE: 5.98828125
RNODES: 4096
REDGES: 12288
RDEGREE: 6
SWI:  0.22360805208643914


'\nbest_net = None\nbest_sur = 0\nworst_net = None\nworst_sur = 10000\n\nfor i in range(100):\n    network = Network(7)\n    network.propogate(12) # 9=512, 10=1024, 11=2048\n    survivability = network.stress_test()\n\n    if survivability > best_sur:\n        best_net = network\n        best_sur = survivability\n\n    if survivability < worst_sur:\n        worst_net = network\n        worst_sur = survivability\n\n#print()\nprint(best_sur)\n#print(worst_sur)\n\nbest_model = NetworkModel(best_net)\nworst_model = NetworkModel(worst_net)\n\nprint(best_model.cluster_co)\nprint(best_model.avg_shortest)\n\nsave_model_data(best_model, \'11-10_10per-of-p2_best-sur\')\n#save_model_data(worst_model,\'11-10_10per-of-p2_worst-sur\')\n\ntotal = len(best_net.graph)\n\nbest_net.remove_top_nodes(best_sur-1)\n#worst_net.remove_top_nodes(worst_sur-1)\n\nbest_per = "{:.2f}".format((best_sur/total)*100)\n#worst_per = "{:.2f}".format((worst_sur/total)*100)\n\nbest_stress_model = NetworkModel(best_net)\n#wo